In [6]:
import requests
import pandas as pd
import numpy as np
from plyer import notification

# Função para exibir alertas
def alerta(titulo, mensagem):
    notification.notify(
        title=titulo,
        message=mensagem,
        app_name="API Data Wrangling",
        timeout=10
    )

# URL da API Rest Countries
url = "https://restcountries.com/v3.1/all"

try:
    # Extração de dados da API
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
    else:
        alerta("Erro na API", f"Código de status {response.status_code}")
        exit()

    # Transformando dados em DataFrame
    df = pd.json_normalize(data)

    # 1. Ajuste de nomes de colunas
    df.columns = [col.lower().replace(".", "_").replace(" ", "_") for col in df.columns]

    # Garantir que as colunas de interesse estejam presentes
    colunas_interesse = [
        "name_common",  # Nome do país
        "region",  # Continente
        "population",  # População
        "languages",  # Línguas
        "currencies"  # Moedas
    ]

    colunas_disponiveis = [col for col in colunas_interesse if col in df.columns]

    # Selecionar apenas as colunas disponíveis
    df = df[colunas_disponiveis]

    # Renomear colunas para nomes mais amigáveis
    df.rename(
        columns={
            "name_common": "country",
            "region": "continent",
            "population": "population",
            "languages": "languages",
            "currencies": "currencies"
        },
        inplace=True
    )

    # 2. Ajuste de valores ausentes
    df["population"] = df["population"].fillna(0)  # Substituir população ausente por 0
    df["continent"] = df["continent"].fillna("Unknown")  # Substituir continente por 'Unknown'

    # 3. Tratamento de 'languages' e 'currencies', caso existam
    if "languages" in df.columns:
        df["languages"] = df["languages"].apply(lambda x: x if isinstance(x, dict) else {})
        df_languages = df.explode("languages")
        df_languages["language"] = df_languages["languages"].apply(lambda x: list(x.values())[0] if isinstance(x, dict) else None)
    else:
        df_languages = pd.DataFrame(columns=["country", "language"])

    if "currencies" in df.columns:
        df["currencies"] = df["currencies"].apply(lambda x: x if isinstance(x, dict) else {})
        df_currencies = df.explode("currencies")
        df_currencies["currency"] = df_currencies["currencies"].apply(lambda x: list(x.keys())[0] if isinstance(x, dict) else None)
    else:
        df_currencies = pd.DataFrame(columns=["country", "currency"])

    # 4. Stack e Unstack
    df_stacked = df.set_index("country").stack()
    df_unstacked = df_stacked.unstack()

    # Exibindo os resultados
    print("Base Tratada:")
    print(df.head())

    print("\nLinguagens Explodidas:")
    print(df_languages.head())

    print("\nMoedas Explodidas:")
    print(df_currencies.head())

    print("\nDados Stack:")
    print(df_stacked.head())

    print("\nDados Unstack:")
    print(df_unstacked.head())

except Exception as e:
    alerta("Erro de Execução", f"Erro ao processar dados: {str(e)}")
    print("Erro de Execução", f"Erro ao processar dados: {str(e)}")

Base Tratada:
         country  continent  population
0  South Georgia  Antarctic          30
1        Grenada   Americas      112519
2    Switzerland     Europe     8654622
3   Sierra Leone     Africa     7976985
4        Hungary     Europe     9749763

Linguagens Explodidas:
Empty DataFrame
Columns: [country, language]
Index: []

Moedas Explodidas:
Empty DataFrame
Columns: [country, currency]
Index: []

Dados Stack:
country                  
South Georgia  continent     Antarctic
               population           30
Grenada        continent      Americas
               population       112519
Switzerland    continent        Europe
dtype: object

Dados Unstack:
               continent population
country                            
South Georgia  Antarctic         30
Grenada         Americas     112519
Switzerland       Europe    8654622
Sierra Leone      Africa    7976985
Hungary           Europe    9749763
